## Pac Learning
1. Assume there is a literal l occurring in h but not in t.
   let p(l) be the probability of picking a positive example with l's negation in the example.
   A literal l is bad if the probability of picking it is p(l) >= epsilon/2n
   where n is the number of atoms in V. thus n is "doubled" because the atoms can be negated.
   if the hypothesis contains no bad literals then: the probability of picking a positive example the hypothesis denies
   is <= 2n(epsilon/2n) = epsilon. Since we assume there are no bad literals in the hypothesis:
   The sum probability of picking a bad literal can't be greater than epsilon (by the definition of a bad literal)

   We know based on the algorithm that the probability of picking a bad literal is the same as the probability of deleting
   it from the hypothesis. Thus the probability of not deleting a bad literal after m calls becomes:
   (1-epsilon/2n)^m since epsilon/2n <= the probability of picking a bad literal.

   Thus the probability of some bad literal not deleted from h after m calls is at least:
   2n(1-epsilon/2n)^m

   Thus we need to find a value of m satisfying 2n(1-epsilon/2n)^m <= delta (1-delta is the desired confidence)
   Solving the inequality for m we get:
   m >= (2n/epsilon)(ln(2n)+ln(1/delta)) since each example takes linear time n to complete;
   and m is polynomial, we know that F is pac learnable in polynomial time.


2. I'm not sure if by all learning frameworks it's meant all possible learning frameworks or all possible learning frameworks
by the definition of task 1. If it's all possible learning frameworks by task 1, it seems pretty intuitive.
We have even shown by task 1. that it is PAC learnable in polynomial time. If it's meant by all possible learning frameworks,
it seems really hard for me to prove. I guess the worst possible scenario for pac learnability is framework where you gain no new information from an example
apart from weather or not your hypothesis was correct for this example. If this is the case, and algorithm would just have to test every possible
hypothesis and until it finds the correct one (since the set of all possible Hypothesises is finite)


In [1]:
import numpy as np
import random

class Literal:
    def __init__(self, value, name):
        self.value = value
        self.name = name
        self.in_target = False
        self.in_hypothesis = True
        self.negation = None

    def __str__(self):
        return str(self.name) + "  " + str(self.value)

    def __repr__(self):
        return str(self.name) + " " + str(self.value)+", "

In [2]:
def generate_literals(n):
    literals = np.empty(2*n, dtype=Literal)
    for i in range(n):
        literals[2*i] = Literal(True, i)
        literals[2*i+1] = Literal(False, i)
        literals[2*i].negation = literals[2*i+1]
        literals[2*i+1].negation = literals[2*i]
        #print(literals[2*i])
        #print(literals[2*i+1])
    return literals

In [3]:
def generate_target(length, literals, distribution, noise, n):
    target = np.empty(length, dtype=Literal)
    for i in range(length):
        contradicting_literal = True
        while contradicting_literal:
            index = random.randint(0, n-1)
            value = distribution[index] + random.randint(-noise, noise)
            if not literals[2*index].in_target and not literals[2*index+1].in_target:
                if value < 50:
                    target[i] = literals[2*index+1]
                    target[i].in_target = True
                    contradicting_literal = False
                else:
                    target[i] = literals[2*index]
                    target[i].in_target = True
                    contradicting_literal = False
    return sorted(target, key= lambda x: x.name)

In [4]:
def generate_example(literals, distribution, noise, n, mean):
    example = np.empty(n, dtype=Literal)
    for i in range (0, n):
        value = distribution[i] + random.randint(-noise, noise)
        if value < mean:
            example[i] = literals[2*i+1]
        else:
            example[i] = literals[2*i]
    return example

In [5]:
def generate_distribution(size, mean, sd):
    #distribution = np.random.binomial(100, 0.5, size)
    distribution = np.random.normal(mean, sd, size)
    return distribution

In [6]:
def positive_example(example):
    pos_example = True
    for i in range(example.size):
        if example[i].negation.in_target:
            pos_example = False
    return pos_example

In [7]:
def update_hypothesis(example):
    if positive_example(example):
        for i in range(example.size):
            example[i].negation.in_hypothesis = False

In [8]:
def print_hypothesis(literals):
    print("Hypothesis: ")
    for literal in literals:
        if literal.in_hypothesis:
            print(literal)

In [9]:
#Returns fraction of correct guesses by hypothesis
def estimate_error(n_error_estimate, literals, distribution, noise, n, mean):
    correct_guesses = 0
    for i in range(n_error_estimate):
        example = generate_example(literals, distribution, noise, n, mean)
        if positive_example(example):
            correct_guess = True
            for j in range(example.size):
                if example[j].negation.in_hypothesis:
                    correct_guess = False
            if correct_guess:
                correct_guesses += 1
        else:
            correct_guesses += 1
    return correct_guesses/n_error_estimate

In [10]:
#It picks a number from the Normal Distribution and assigns it to a literal
#If the number is more than the mean the probability of picking the positive literal increases in examples and target
#Noise is added so it will be a probability(instead of certainty) based on the normal distribution weather or not it picks the positive or negative literal
#Thus higher SD will mean less "noise"

#I decided to assign req_size by the definition from the book how I understood it (Of course I may be wrong)
#If I used the req_size definition from the obligatory assignment and a target of one literal;
#I was unable to meet the generalization guarantee.
def main():
    #I recommend using 50 as mean, and instead tweaking noise and SD in order to generate different types of data
    n = int(input("Enter number of PL-atoms"))
    epsilon = float(input("Enter Epsilon"))
    delta = float(input("Enter Delta"))
    mean = float(input("Enter mean of Normal Distribution"))
    sd = float(input("Enter SD (spread or “width”) of the distribution."))

    noise = 10
    target_size = 10
    n_error_estimate = int((20/epsilon)**2)
    error_greater_than_epsilon_count = 0
    #Required size to have less than epsilon error with 1-delta confidence
    req_size = int(((2*n)/epsilon)*(np.log(2*n)+np.log(1/delta)))
    #req_size = int(2/(epsilon*(np.log(2)+np.log(1/delta))))
    #print("Required Size: ", req_size)
    #print(10/delta)
    for i in range(int(10/delta)):
        literals = generate_literals(n)
        distribution = generate_distribution(n, mean, sd)
        target = generate_target(target_size, literals,distribution, noise, n)

        #print("Distribution:\n",distribution)
        print("Target:\n", target)

        for j in range (req_size):
            example = generate_example(literals, distribution, noise, n, mean)
            update_hypothesis(example)

        print_hypothesis(literals)
        accuracy = estimate_error(n_error_estimate, literals, distribution, noise, n, mean)
        print("Estimated Accuracy ", accuracy)
        if accuracy < (1-epsilon):
            error_greater_than_epsilon_count += 1
    print("Number of errors greater than 10? number of errors:",error_greater_than_epsilon_count)

main()

Target:
 [1 True, , 3 True, , 4 True, , 6 True, , 9 False, , 12 False, , 14 True, , 15 False, , 17 True, , 19 True, ]
Hypothesis: 
1  True
3  True
4  True
5  True
6  True
9  False
12  False
14  True
15  False
17  True
19  True
Estimated Accuracy  1.0
Target:
 [0 True, , 2 False, , 4 False, , 5 True, , 8 False, , 9 True, , 10 True, , 14 False, , 15 True, , 18 False, ]
Hypothesis: 
0  True
2  False
4  False
5  True
8  False
9  True
10  True
14  False
15  True
18  False
Estimated Accuracy  1.0
Target:
 [0 False, , 7 False, , 9 False, , 10 True, , 13 True, , 14 True, , 15 True, , 16 True, , 18 False, , 19 True, ]
Hypothesis: 
0  False
7  False
9  False
10  True
13  True
14  True
15  True
16  True
18  False
19  True
Estimated Accuracy  1.0
Target:
 [0 True, , 2 False, , 4 False, , 6 False, , 9 True, , 12 False, , 13 False, , 16 False, , 17 True, , 18 True, ]
Hypothesis: 
0  True
2  False
4  False
6  False
7  True
9  True
12  False
13  False
16  False
17  True
18  True
19  False
Estimated Ac

KeyboardInterrupt: 